In [ ]:
# Import packages
## Prepare and explore data
import pandas as pd # Data manipulation
import numpy as np # Numeric computations
import matplotlib.pyplot as plt # Data visualisation
import seaborn as sns # Data visualisation

## Machine learning packages
from sklearn.model_selection import train_test_split # Split training and test data
from sklearn.metrics import mean_absolute_error, mean_squared_error # Model evaluation
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet #Linear models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor # Ensemble models
from sklearn.svm import SVC # Support vector machine
from sklearn.naive_bayes import GaussianNB # Naive Bayes
from sklearn.tree import DecisionTreeRegressor # Decision Tree
from xgboost import XGBRegressor # Extreme gradient boostng
from category_encoders import CategoricalEncoder # One hot encoder
from sklearn.pipeline import Pipeline, make_pipeline # Pipeline

# Prepare Data
## Import 

In [ ]:
df = pd.read_csv("heigh_prediction.csv")

## Explore

## Split

# Build model
## Baseline

## Iterate

# Evaluate

# Communicate